In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
from ase.build import fcc111, molecule, fcc100, fcc110
from ase.visualize import view
from ase import Atoms
from ase.io import write

In [3]:
from ase.optimize import BFGS, FIRE
from ase.constraints import ExpCellFilter

# Pt surface

In [4]:
el='Pt'

In [5]:
# Build an FCC (100) surface
surface = fcc100(el, size=(4, 4, 6), vacuum=20.0)

In [6]:
# Create an O2 molecule
o2 = molecule('O2')
o2.rotate([0,0,1],[1,0,0])

z_max=max(surface.positions[:,2])

# Adjust the position of the O2 molecule above the surface
o2.translate([surface.get_cell()[0, 0] / 2, surface.get_cell()[1, 1] / 2, z_max+2.5])

In [7]:
# Combine the surface and the O2 molecule into a single system
system = surface + o2

# Visualize the system (optional)
# view(system)

In [8]:
fname=f"{el}_surface-fcc-slab_and_O2.lammps-data"
fname

'Pt_surface-fcc-slab_and_O2.lammps-data'

In [9]:
specorder=[el,'O']
specorder

['Pt', 'O']

In [10]:
write(fname, system, specorder=specorder)

# Ethanol water

In [4]:
from ase.build import molecule
from ase import Atoms

In [5]:
from matscipy.neighbours import neighbour_list

In [82]:
water=molecule("H2O")

In [83]:
water.get_chemical_symbols()

['O', 'H', 'H']

In [85]:
water.set_chemical_symbols(['Li','He','He'])

In [86]:
ethanol=molecule("CH3CH2OH")

In [87]:
def insert_molecules(tot_cell, mol, N_mol, MIN_THRESHOLD ):
    
    for _ in range(N_mol):

        while True:
            new_mol=mol.copy()
            new_mol.rotate([0,0,1], np.random.randn(3))

            rnd_shift=np.random.rand(3)*np.diag(tot_cell.cell)

            new_mol.positions+=rnd_shift

            new_tot_cell = tot_cell+new_mol

            min_dist_list=neighbour_list("ijd", new_tot_cell, cutoff=3.0)
            ii,jj,dd=min_dist_list

            new_indices = np.arange(len(new_tot_cell))[-len(new_mol):]
            old_indices = np.arange(len(new_tot_cell))[:-len(new_mol)]
#             print(f"{new_indices=}")
#             print(f"{old_indices=}")

            mask=np.isin(ii, new_indices) & np.isin(jj, old_indices)

            if np.any(mask):
                min_dist = np.min(dd[mask])
            else:
                min_dist = 100


            if min_dist >= MIN_THRESHOLD:
                break

        tot_cell = new_tot_cell                
        print(f"Current structure: num.at={len(tot_cell)}, min_dist={min_dist}, overall min = {np.min(dd)}")        
    return tot_cell


In [88]:
MIN_THRESHOLD=2

In [89]:
tot_cell = ethanol.copy()
tot_cell.set_cell([15,15,15])
tot_cell.set_pbc(True)

In [90]:
tot_cell_1=insert_molecules(tot_cell, ethanol, 10, MIN_THRESHOLD=MIN_THRESHOLD)

Current structure: num.at=18, min_dist=100, overall min = 0.971323891004436
Current structure: num.at=27, min_dist=100, overall min = 0.971323891004436
Current structure: num.at=36, min_dist=2.1269255479101425, overall min = 0.971323891004436
Current structure: num.at=45, min_dist=2.4383368473708695, overall min = 0.971323891004436
Current structure: num.at=54, min_dist=2.7246965298040458, overall min = 0.971323891004436
Current structure: num.at=63, min_dist=100, overall min = 0.971323891004436
Current structure: num.at=72, min_dist=100, overall min = 0.971323891004436
Current structure: num.at=81, min_dist=100, overall min = 0.971323891004436
Current structure: num.at=90, min_dist=2.370196415390373, overall min = 0.971323891004436
Current structure: num.at=99, min_dist=2.356404020862197, overall min = 0.971323891004436


In [100]:
#view(tot_cell_1)

In [92]:
tot_cell_2=insert_molecules(tot_cell_1, water, 49, MIN_THRESHOLD=MIN_THRESHOLD)

Current structure: num.at=102, min_dist=100, overall min = 0.9685650182625833
Current structure: num.at=105, min_dist=100, overall min = 0.9685650182625833
Current structure: num.at=108, min_dist=2.2574076598579533, overall min = 0.9685650182625833
Current structure: num.at=111, min_dist=2.1208914883685273, overall min = 0.9685650182625833
Current structure: num.at=114, min_dist=2.0260809184172266, overall min = 0.9685650182625833
Current structure: num.at=117, min_dist=2.436853715342068, overall min = 0.9685650182625833
Current structure: num.at=120, min_dist=100, overall min = 0.9685650182625827
Current structure: num.at=123, min_dist=100, overall min = 0.9685650182625827
Current structure: num.at=126, min_dist=2.041145053660059, overall min = 0.9685650182625827
Current structure: num.at=129, min_dist=2.985272215446766, overall min = 0.9685650182625827
Current structure: num.at=132, min_dist=2.7321382828959506, overall min = 0.9685650182625827
Current structure: num.at=135, min_dist=

In [93]:
#view(tot_cell_2)

<Popen: returncode: None args: ['/fscratch/conda/tp2/bin/python', '-m', 'ase...>

In [94]:
conv_factor=1.6605402e-27 / 1e-30# amu/A3 -> kg/m3

In [95]:
sum(tot_cell_2.get_masses())/sum(tot_cell_2.get_volume())*conv_factor

609.6391647800416

In [96]:
len(tot_cell_2)

246

In [97]:
write("2-ethanol-water/ethanol-water.lammps-data", tot_cell_2, format='lammps-data', specorder=['C','H','O','He', 'Li'])

In [98]:
min_dist_list=neighbour_list("ijd", tot_cell_1,cutoff=3.0)

In [99]:
np.min(min_dist_list[2])

0.971323891004436